In [ ]:
import pandas as pd
import os
import re
import html
import string

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [ ]:
splitSize = 80000
filePath = '/FILEPATH/FILE.csv'

for i, split in enumerate(pd.read_csv(filePath, chunksize = splitSize, encoding='ISO-8859-1')):
    split.to_csv(f'testSplit{i}.csv', index = False)

In [ ]:
filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    if os.path.exists(fileName):
        df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
        if df.shape[1] >= 3:
            uniqueVal = df.iloc[:,3].unique()
            print(f'uniques - {fileName}: {uniqueVal}')
        else:
            print(f'{fileName} has <3 col')
    else:
        print(f'{fileName} DNE')

##we know from this that all the files exist and that the loop works as intended
##therefore, we don't need any more else statements

In [25]:
filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
    df = df.iloc[:,[0,5]]
    df.to_csv(fileName, index = False)
    
##clearing out all the extraneous columns (indices 1-4)
##keeping [TARGET] and [MESSAGE] columns only :)

In [26]:
##now we need to replace all @ mentions in all files with generic "@user" for
##the sake of efficiency

filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
    ##define @username pattern
    userPattern = r'@\w+'
    df.iloc[:,1] = df.iloc[:,1].str.replace(userPattern, '@user', regex = True)
    df.to_csv(fileName, index = False)
    
##all files should be ready to go...

In [27]:
##now we will be removing all urls and encoding errors:

def clean(line):
    line = html.unescape(line)
    line = re.sub(r'[^\x00-\x7F]+', ' ', line)
    line = re.sub(r'http\S+|www\S+|https\S+', '', line, flags = re.MULTILINE)
    return line

filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
    df.iloc[:,1] = df.iloc[:,1].apply(clean)
    df.to_csv(fileName, index = False)
    
##done yayyy

In [ ]:
##checking if neutral (value of 2 in col index 0) exists
filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
    uniqueVal = df.iloc[:,0].unique()
    print(f'uniques - {fileName}: {uniqueVal}')
    
##we know from this that all the files exist and that the loop works as intended
##therefore, we don't need any more else statements

In [ ]:
##checking source file for uniques

filePath = '/FILEPATH/FILE.csv'
df = pd.read_csv(filePath, encoding = 'ISO-8859-1')
uniqueVal = df.iloc[:,0].unique()
print(f'uniques - {fileName}: {uniqueVal}')

##as 2 (neutral) doesnt exist in my cleaned data

In [6]:
##checking average length of sentence (by word)

filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20
##define globalavg
globalAvg = 0

##main loop
for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
    ##action
    sentenceCol = df.iloc[:,1]
    wordCount = sentenceCol.str.split().apply(len)
    avg = wordCount.mean()
    globalAvg += avg

##get and print global avg length of sentence
globalAvg /= 20
print(f'avg: {globalAvg}')
  
## avg: 13.131260473709045
## now to pad and cut :,) 

##ran this again after the next block to check if sentences are fixed length
##the average is 13.0. they are fixed yay


avg: 13.0


In [28]:
globalAvg = 13

filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')    
    ##define sentence column
    sentenceCol = df.iloc[:,1]

    ##function to delete extra words
    def fixedlength(sentence):
        words = sentence.split()
        if len(words) > globalAvg:
            words = words[:globalAvg]
        return ' '.join(words)
    
    ##save df
    df.iloc[:,1] = sentenceCol.apply(fixedlength)
    df.to_csv(fileName, index = False)


In [29]:
##removing all nan

##filepath
filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20
    
for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
    df.dropna(inplace = True)
    ##save df
    df.to_csv(fileName, index = False)


In [30]:
##urls not removed for some reason so we go again ig
def removeURLfr(sentence):
    urlPattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(urlPattern, ' ', sentence)

##filepath
filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding='ISO-8859-1')
    ##col ind1
    sentenceCol = df.iloc[:, 1]
    df.iloc[:, 1] = sentenceCol.apply(removeURLfr)
    df.to_csv(fileName, index=False)

In [31]:
##remove all special characters and uppercase

def removeBad(sentence):
    specialChar = string.punctuation.replace('@', '')
    key = ' ' * len(specialChar)
    translator = str.maketrans(specialChar, key)
    cleaned = sentence.translate(translator).lower()
    return cleaned

##function to pad or delete extra words
def fixedlength(sentence):
    words = sentence.split()
    if len(words) < globalAvg:
        words.extend(['<PAD>'] * (globalAvg - len(words)))
    elif len(words) > globalAvg:
        words = words[:globalAvg]
    return ' '.join(words)

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding='ISO-8859-1')
    ##col ind1
    sentenceCol = df.iloc[:, 1]
    df.iloc[:,1] = sentenceCol.apply(removeBad)
    df.iloc[:,1] = sentenceCol.apply(fixedlength)
    df.to_csv(fileName, index=False)

In [35]:
##checking global average length of sentence again

filePath = '/FILEPATH/'
filePrefix = 'testSplit'
fileCount = 20
globalAvg = 0

for i in range(fileCount):
    fileName = f'{filePath}{filePrefix}{i}.csv'
    df = pd.read_csv(fileName, encoding = 'ISO-8859-1')
    ##action
    sentenceCol = df.iloc[:,1]
    wordCount = sentenceCol.str.split().apply(len)
    avg = wordCount.mean()
    globalAvg += avg
    
globalAvg /= 20
print(f'avg: {globalAvg}')

##we get 13.0, which is our target :)

avg: 13.0


In [5]:
import numpy as np

In [7]:
##sorting 0-10000 for dataset 0 and 19 randomly, as it contains both +/- sentiment

filePath1 = '/FILEPATH/FILE.csv'
filePath2 = '/FILEPATH/FILE.csv'
df1 = pd.read_csv(filePath1)
df2 = pd.read_csv(filePath2)
df1Head = df1.head(10000)
df2Head = df2.head(10000)
df = pd.concat([df1Head, df2Head], axis = 0)
df = df.sample(frac = 1, random_state = 7)
newFilePath = '/FILEPATH/FILE.csv'
df.to_csv(newFilePath, index = False, header = False)


In [4]:
from gensim.models import KeyedVectors
modelPath = '/FILEPATH/FILE.txt'
wordVec = KeyedVectors.load_word2vec_format(modelPath, binary=False, no_header = True)

In [8]:
##now we must embed every sentence in our new dataset
##wordvec
#modelPath = '/Users/aly/Desktop/Main/funni_code/mldata/glove.6B.50d.txt'
#wordVec = KeyedVectors.load_word2vec_format(modelPath, binary=False, no_header = True)

def sentenceEmbed(sentence, wordVec):
    words = sentence.split()
    wordEmbedding = [wordVec[word] for word in words if word in wordVec]
    if len(wordEmbedding) == 0:
        return None
    sentenceEmbed = np.mean(wordEmbedding, axis=0)
    return sentenceEmbed

filePath = '/FILEPATH/FILE.csv'
df = pd.read_csv(filePath)
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: sentenceEmbed(x, wordVec))
df.to_csv(filePath, index = False, header = False)

In [7]:
filePath1 = '/FILEPATH/FILE.csv'
filePath2 = '/FILEPATH/FILE.csv'
df1 = pd.read_csv(filePath1)
df2 = pd.read_csv(filePath2)


df1Head = df1.head(50000)
df2Head = df2.head(50000)
df = pd.concat([df1Head, df2Head], axis = 0)
df = df.sample(frac = 1, random_state = 7)
newFilePath = '/FILEPATH/FILE.csv'
df.to_csv(newFilePath, index = False, header = False)

def sentenceEmbed(sentence, wordVec):
    words = sentence.split()
    wordEmbedding = [wordVec[word] for word in words if word in wordVec]
    if len(wordEmbedding) == 0:
        return None
    sentenceEmbed = np.mean(wordEmbedding, axis=0)
    return sentenceEmbed

df.iloc[:,1] = df.iloc[:,1].apply(lambda x: sentenceEmbed(x, wordVec))
df.to_csv(newFilePath, index = False, header = False)